# Poly model
Working on a version of `design_linear.py` that takes a numpy  Polynomial class (or a list of coefficients) instead of a single value to apply linaerly as m * x + 0.0

Also trying to evaluate polynominal over whole array not point.

In [78]:
%load_ext line_profiler

In [2]:
import numpy as np
from numpy.polynomial import Polynomial

import cmm_error_map.design_poly as designp
import cmm_error_map.design_linear as designl
import cmm_error_map.data_cmpts as dc
import cmm_error_map.config.config as cf

In [6]:
model_params = dc.model_parameters_test.copy()


In [17]:
# create polys from params
coeffs = {
    key: value if isinstance(value, list) else [0.0, value]
    for key, value in model_params.items()
}
model_polys = {key: Polynomial(coeffs[key]) for key in model_params}

In [65]:
def model_array(
    xyz_in: np.ndarray,  # (3, n)
    xyzt: np.ndarray,  # (3,)
    model_polys: dict[str, Polynomial],
    fixed_table=False,
    bridge_axis=1,
):
    """ """
    if not fixed_table:
        # table movement is in opposite direction to probe position for a moving table
        dirn = [1, 1, 1]
        table_axis = int(not bridge_axis)
        dirn[table_axis] = -1
        xyz = (xyz_in.T * dirn).T
    else:
        xyz = xyz_in

    x, y, z = xyz
    dep_dict = {
        "Txx": x,
        "Txy": x,
        "Txz": x,
        "Tyx": y,
        "Tyy": y,
        "Tyz": y,
        "Tzx": z,
        "Tzy": z,
        "Tzz": z,
        "Rxx": x,
        "Rxy": x,
        "Rxz": x,
        "Ryx": y,
        "Ryy": y,
        "Ryz": y,
        "Rzx": z,
        "Rzy": z,
        "Rzz": z,
        "Wxy": x,
        "Wxz": y,
        "Wyz": z,
    }
    poly_evals = {}
    for key in dep_dict:
        poly_evals[key] = model_polys[key](dep_dict[key])
    pl = poly_evals
    # poly_evals = {key: model_polys[key](dep_dict[key]) for key in dep_dict}
    dev_out = np.empty_like(xyz_in)
    for i in range(xyz_in.shape[1]):
        rxl = np.array(
            [
                [1.0, pl["Rxz"][i], -pl["Rxy"][i]],
                [-pl["Rxz"][i], 1.0, pl["Rxx"][i]],
                [pl["Rxy"][i], -pl["Rxx"][i], 1.0],
            ]
        )
        ryl = np.array(
            [
                [1.0, pl["Ryz"][i], -pl["Ryy"][i]],
                [-pl["Ryz"][i], 1.0, pl["Ryx"][i]],
                [pl["Ryy"][i], -pl["Ryx"][i], 1.0],
            ]
        )
        rzl = np.array(
            [
                [1.0, pl["Rzz"][i], -pl["Rzy"][i]],
                [-pl["Rzz"][i], 1.0, pl["Rzx"][i]],
                [pl["Rzy"][i], -pl["Rzx"][i], 1.0],
            ]
        )
        # inverse of a rotation matrix is its transpose
        inv_rxl = rxl.T
        inv_ryl = ryl.T
        inv_rzl = rzl.T
        # translation
        x, y, z = xyz[:, i]
        xl = np.array([x + pl["Txx"][i], pl["Txy"][i], pl["Txz"][i]])
        yl = np.array([pl["Tyx"][i], y + pl["Tyy"][i], pl["Tyz"][i]])
        zl = np.array([pl["Tzx"][i], pl["Tzy"][i], z + pl["Tzz"][i]])
        # @ is matrix multiplication, fmt:skip keeps formatting when using ruff
        if fixed_table and bridge_axis == 0:
            # moving bridge, x axis across bridge - eg Hexagon ... (Bruce)
            w = (
                + yl
                + inv_ryl @ xl
                + inv_ryl @ inv_rxl @ zl
                + inv_ryl @ inv_rxl @ inv_rzl @ xyzt
            )  # fmt: skip

        elif fixed_table and bridge_axis == 1:
            # moving bridge y-axis across bridge - eg ? - for completeness
            w = (
                + xl
                + inv_rxl @ yl
                + inv_rxl @ inv_ryl @ zl
                + inv_rxl @ inv_ryl @ inv_rzl @ xyzt
            )  # fmt: skip
        elif not fixed_table and bridge_axis == 0:
            # fixed brige, moving table, x-axis across bridge eg. Mitutoyo  (Shishimi)
            w =  (
                - ryl @ yl 
                + ryl @ xl 
                + ryl @ inv_rxl @ zl 
                + ryl @ inv_rxl @ inv_rzl @ xyzt
            )  # fmt: skip
        elif not fixed_table and bridge_axis == 1:
            # fixed brige, moving table, y-axis across bridge eg. old PMM866
            w = (
                - rxl @ xl 
                + rxl @ yl 
                + rxl @ inv_ryl @ zl 
                + rxl @ inv_ryl @ inv_rzl @ xyzt
            )  # fmt: skip
        else:
            raise ValueError("Unknown cmm type")

        dev3d = w - xyz_in[:, i] - xyzt
        dev_out[:, i] = dev3d

    return dev_out


In [86]:
ballspacing = 133.0
x0, y0, z0 = 250.0, 50.0, 50.0
# XY plane
transform_mat_xy = dc.matrix_from_vectors([x0, y0, z0], [0.0, 0.0, 0.0])
xt, yt, zt = 0.0, 0.0, -243.4852
prb_xy = dc.Probe(title="P0", name="p0", length=np.array([xt, yt, zt]))
mmt_xy = dc.Measurement(
    title="Plate XY",
    name="mmt_00",
    artefact=cf.artefact_models["KOBA 0620"],
    transform_mat=transform_mat_xy,
    probe=prb_xy,
    cmm_nominal=None,
    cmm_dev=None,
    mmt_nominal=None,
    mmt_dev=None,
)
mmt_xy.recalculate(model_params, dc.pmm_866.cmm_model)

In [66]:
dev_out = model_array(mmt_xy.cmm_nominal, mmt_xy.probe.length, model_polys)
dev_out.T

array([[ 0.00340559,  0.00063631, -0.00086722],
       [ 0.00474494,  0.00125001, -0.00211394],
       [ 0.0060843 ,  0.0020406 , -0.00390549],
       [ 0.00742365,  0.00300808, -0.00624187],
       [ 0.008763  ,  0.00415245, -0.00912308],
       [ 0.00211773, -0.00015851, -0.00061817],
       [ 0.00336864,  0.00045518, -0.0017324 ],
       [ 0.00461955,  0.00124577, -0.00339146],
       [ 0.00587046,  0.00221325, -0.00559535],
       [ 0.00712137,  0.00335762, -0.00834406],
       [ 0.00082987, -0.00095334, -0.00036913],
       [ 0.00199233, -0.00033965, -0.00135086],
       [ 0.0031548 ,  0.00045094, -0.00287743],
       [ 0.00431726,  0.00141842, -0.00494882],
       [ 0.00547973,  0.00256279, -0.00756504],
       [-0.00045799, -0.00174817, -0.00012008],
       [ 0.00061603, -0.00113447, -0.00096932],
       [ 0.00169005, -0.00034389, -0.0023634 ],
       [ 0.00276407,  0.00062359, -0.0043023 ],
       [ 0.00383809,  0.00176796, -0.00678603],
       [-0.00174585, -0.002543  ,  0.000

In [83]:
designp.model_matrix(mmt_xy.cmm_nominal, mmt_xy.probe.length, model_params)

array([[ 0.00340559,  0.00063631, -0.00086722],
       [ 0.00474494,  0.00125001, -0.00211394],
       [ 0.0060843 ,  0.0020406 , -0.00390549],
       [ 0.00742365,  0.00300808, -0.00624187],
       [ 0.008763  ,  0.00415245, -0.00912308],
       [ 0.00211773, -0.00015851, -0.00061817],
       [ 0.00336864,  0.00045518, -0.0017324 ],
       [ 0.00461955,  0.00124577, -0.00339146],
       [ 0.00587046,  0.00221325, -0.00559535],
       [ 0.00712137,  0.00335762, -0.00834406],
       [ 0.00082987, -0.00095334, -0.00036913],
       [ 0.00199233, -0.00033965, -0.00135086],
       [ 0.0031548 ,  0.00045094, -0.00287743],
       [ 0.00431726,  0.00141842, -0.00494882],
       [ 0.00547973,  0.00256279, -0.00756504],
       [-0.00045799, -0.00174817, -0.00012008],
       [ 0.00061603, -0.00113447, -0.00096932],
       [ 0.00169005, -0.00034389, -0.0023634 ],
       [ 0.00276407,  0.00062359, -0.0043023 ],
       [ 0.00383809,  0.00176796, -0.00678603],
       [-0.00174585, -0.002543  ,  0.000

In [74]:
%timeit model_array(mmt_xy.cmm_nominal, mmt_xy.probe.length, model_polys)

2.04 ms ± 171 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [84]:
%timeit designp.model_matrix(mmt_xy.cmm_nominal, mmt_xy.probe.length, model_params)

1.72 ms ± 117 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [85]:
%lprun -f designp.model_matrix designp.model_matrix(mmt_xy.cmm_nominal, mmt_xy.probe.length, model_params)

Timer unit: 1e-09 s

Total time: 0.00343113 s
File: /home/elfnor/gits/ls_cmm_error_map/src/cmm_error_map/design_poly.py
Function: model_matrix at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                           def model_matrix(
    15                                               xyz_in: np.ndarray,  # (3, n)
    16                                               xyzt: np.ndarray,  # (3,)
    17                                               model_params: dict[str, float | list[float]],
    18                                               fixed_table=False,
    19                                               bridge_axis=1,
    20                                           ):
    21                                               """ """
    22                                               # create polys from params
    23         2      48599.0  24299.5      1.4      coeffs = {
    24                                                   

In [73]:
d = designl.linear_model_matrix(mmt_xy.cmm_nominal.T, mmt_xy.probe.length, model_params)
d.shape

(25, 3)